In [ ]:
# rnn 
- 순서와 시간성을 가진데이터에서 특징을 추출하는 망 
- rnn 을 구성하는 cell 하나는 ffnn망으로 구성된다.
- cell 과 cell 을 연결해서 시간적으로 누적되는 특성을 추출한다.
- rnn은 다양한 망 구성이 가능 
    - one to one  : 바닐라망 (기본망)
    - one to many : captioning
    - many to one : 감정분석
    - many to many : 번역망이나 비디오 해석


In [9]:
import numpy as np 
import os 
import tensorflow as tf 
tf.reset_default_graph()
n_inputs = 3 # 셀당 입력차수
n_neurons = 5 # 셀당 출력차수
X0 = tf.placeholder(tf.float32 , [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None,n_inputs])

#  셀정의
# 셀당 필요한 가중치 (3 x 5)
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units= n_neurons) # 출력차수 

# rnn 망 구성  static_rnn(메모리를 확보), dynamic_rnn(동적으로 메모리를 확보)

output_seqs, states = tf.contrib.rnn.static_rnn(
    basic_cell,[X0,X1], dtype=tf.float32) # 2 x 4 x 3  X0,X1  batch_size = 2  , 셀수 : 4 

# 2 x 4 x 3 * 3 x 5  =   2 x 4 x 5  

# output_seqs = 2 x 4 x 5 
# states = 1 x 5 

Y0, Y1 = output_seqs
init = tf.global_variables_initializer()
# 번역: 나는 학교에 간다 > encoding
# A ASCIL CODE : A = 65 , 

X0_batch = np.array([[0,1,2],[3,4,5],[6,7,8],[9,0,1]])  # 입력 4 x 3 
X1_batch = np.array([[9,8,7,],[0,0,0],[6,5,4],[3,2,1]])
with tf.Session() as sess:
    init.run()
    Y0_val , Y1_val = sess.run([Y0,Y1], feed_dict={X0: X0_batch, X1: X1_batch}) # ㅊ리결과 나온 sequence
    
print("처음 데이터 특성:" , Y0_val,"차수 :" , Y0_val.shape)
print("처음 데이터 특성:" , Y1_val,"차수 :" , Y1_val.shape)

[<tf.Tensor 'rnn/basic_rnn_cell/Tanh:0' shape=(?, 5) dtype=float32>, <tf.Tensor 'rnn/basic_rnn_cell/Tanh_1:0' shape=(?, 5) dtype=float32>]
처음 데이터 특성: [[-0.86169916  0.8432061   0.47443736 -0.34020853  0.91262394]
 [-0.954275    0.97413385 -0.19946632 -0.9837568   0.9999926 ]
 [-0.9853696   0.99597186 -0.7259611  -0.99972767  1.        ]
 [ 0.99996036 -0.9998853  -0.99974436 -0.9349524   0.9999246 ]] 차수 : (4, 5)
처음 데이터 특성: [[-0.80336785  0.96861356 -0.9838191  -0.9998541   1.        ]
 [-0.35039443  0.29315513 -0.27847555  0.65464085 -0.44384304]
 [ 0.1010991   0.6439424  -0.9837956  -0.99629325  0.9999995 ]
 [ 0.6921092  -0.82303387 -0.85478467 -0.9696164   0.97691935]] 차수 : (4, 5)


In [10]:
# 다음 모델에서 차수들을 결정하시요

In [15]:
import numpy as np 
import tensorflow as tf
tf.reset_default_graph()
n_steps = 2 
n_inputs = 3 
n_nerouns = 5
X =tf.placeholder(tf.float32,[None, n_steps, n_inputs])

basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
# static : for 문을 사용하지 않고 메모리 공간확보
# dynamic : for 문을 이용해서 계산
outputs , states = tf.nn.dynamic_rnn(basic_cell,X,dtype = tf.float32)

#outputs = 4 x 2 x 5 ,
# states = 1 x 5 

X_batch = np.array([
                    [[0,1,2],[9,8,7]],
                   [[3,4,5],[3,4,5]],
                   [[6,7,8],[6,5,4]],
                   [[9,0,1],[3,2,1]],
                   ])   # 4 x 4 x 3 

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    outputs_val = outputs.eval(feed_dict={X:X_batch})
print("outputs_val:{}\n{}".format(outputs_val.shape,outputs_val))


outputs_val:(4, 2, 5)
[[[-0.54625416  0.6476071  -0.7252678  -0.6264582  -0.66258126]
  [ 0.8503437   0.9594502  -0.96957827  0.98127    -0.9874417 ]]

 [[-0.25112727  0.9328735  -0.9675937  -0.5148283  -0.9565312 ]
  [-0.68045765  0.9429903  -0.9543268   0.5114724  -0.954179  ]]

 [[ 0.09946536  0.98878515 -0.99659896 -0.38250393 -0.9951475 ]
  [ 0.6233073   0.8172634  -0.89223623  0.96385914 -0.8918634 ]]

 [[ 0.9820139   0.9573325   0.9976685   0.54451734 -0.83294356]
  [ 0.02791571  0.2852857  -0.80430776  0.1067356   0.27423617]]]


In [21]:
import matplotlib.pyplot as plt 
tf.reset_default_graph() # 그래프 초기화
n_steps = 28 #셀수 
n_inputs = 28  #셀당 입력수 28 x 28 = image한장
n_neurons = 150 # 셀당 출력수 (특징확대)
n_outputs = 10  # 150개의 추출 > 10 개로 축소
learning_rate = 0.001

X = tf.placeholder(tf.float32,[None,n_steps,n_inputs]) # ? x 28 x 28
y = tf.placeholder(tf.int32,[None]) # ?
# RNN 망 구성 및 특징 추출 

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons) # 28 > 150 
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32) # ? ,28 ,28

# outputs :  150 x 28 x 150
# states : 150 x 150
print(outputs, states)
# FFNN 시작

logits = tf.layers.dense(states, n_outputs) #fully-connected : 가중치 생략
# 가중치를 계산 : 150  x 150     150  x 10  
# cost function : cross_entropy 
# 확률값으로 변환 
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels= y , logits=logits)

loss = tf.reduce_mean(xentropy) # 배치사이즈에 대한 평균값으로 = 비용값
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(loss)  # = 학습
correct = tf.nn.in_top_k(logits, y , 1) # 가장 큰 수의 인덱스를 구하고 실제값과 비교
accuracy = tf.reduce_mean(tf.cast(correct,tf.float32)) # 확률값은 소수점까지 계산 > float32  = 정확도
init = tf.global_variables_initializer()



Tensor("rnn/transpose_1:0", shape=(?, 28, 150), dtype=float32) Tensor("rnn/while/Exit_3:0", shape=(?, 150), dtype=float32)


In [22]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")
X_test = mnist.test.images.reshape((-1 , n_steps, n_inputs)) # 원래의 이미지 사이즈 
y_test = mnist.test.labels
n_epochs = 100 
batch_size = 150 

with tf.Session() as sess:
    # 60000/150
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples//batch_size):
            X_batch , y_batch = mnist.train.next_batch(batch_size)
            # -1 : 계산결과 나머지 : 150
            X_batch  = X_batch.reshape((-1, n_steps, n_inputs))
            # 150x28x28
            sess.run(training_op , feed_dict={X:X_batch, y: y_batch}) # 학습
        acc_train = accuracy.eval(feed_dict={X:X_batch, y: y_batch}) # 정확도 계산
        acc_test = accuracy.eval(feed_dict={X:X_test, y: y_test})
        print(epoch, "Train accuracy : ", acc_train, "Test accuracy:", acc_test)
        






Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
0 Train accuracy :  0.8933333 Test accuracy: 0.918
1 Train accuracy :  0.97333336 Test accuracy: 0.9471
2 Train accuracy :  0.97333336 Test accuracy: 0.9583
3 Train accuracy :  0.96666664 Test accuracy: 0.9565
4 Train accuracy :  0.96666664 Test accuracy: 0.96
5 Train accuracy :  0.97333336 Test accuracy: 0.9616
6 Train accuracy :  0.97333336 Test accuracy: 0.9632

In [57]:
# encoding > decoding
# 끝단어 예측
tf.reset_default_graph()
import tensorflow as tf
import numpy as np
char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g',
            'h', 'i', 'j', 'k', 'l', 'm', 'n',
            'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']

# 글자에 대한 인덱스 번호 구성 
num_dic = {n: i for i, n in enumerate(char_arr)} # enumerate 

print(num_dic) # num_dic > 문자 순서
# 'a': 0, 'b': 1, 'c': 2, 'd': 3,

dic_len = len(num_dic)

seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind'] 


{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25}


In [74]:
# 원핫 인코딩
# 셀당 입력 26  a ~ z 
# 셀개수 3 > w , o , r 
def make_batch(seq_data):
    input_batch = [] # 3글자 
    target_batch = [] # 1글자
    for seq in seq_data: #  word
        input=[num_dic[n] for n in seq[:-1]] # wor -> w : 22 , o : 14 ,r:17  > encoding
        target = num_dic[seq[-1]] # d : 3 
        #신경망에서 범주형 데이터 one-hot-encoding
        # eye: 단위행렬
        input_batch.append(np.eye(dic_len)[input]) # 원핫인코딩된 wor 값을 append
        target_batch.append(target) # 원핫인코딩된 d 값을 append        
    return input_batch, target_batch
     

In [75]:

learning_rate = 0.01
n_hidden = 128  # 가중치를 위한 히든 / 셀당 나가는 차수
total_epoch = 30
n_step = 3 # 셀수 3 w,o,r 개 
n_input = n_class = dic_len # 26(a~z)
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.int32, [None])
# FFNN을 위한 가중치 준비

W = tf.Variable(tf.random_normal([n_hidden,n_class])) # 배치사이즈 , 셀수 , 셀당입력수
b = tf.Variable(tf.random_normal([n_class])) #128 x 26
# RNN으로 특징을 추출 (셀당 가중치 학습)
# control_state 를 추가 : 장기 기억이 가능하도록 : 변종들이 많이 있음

cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5, seed=10) # 계산회로를 랜덤하게 줄임 (과적합) 현재 50프로만 계산
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2]) # 상하로 멀티 셀을 쌓는 망 ( 더 정확하게 학습하기 위해서 # ? x 3 x 26
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)
 # 3 x 배치사이즈 x 128
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]

model = tf.matmul(outputs, W) + b  # Dense 하고 동일하게 연산  배치사이즈 x 26
cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=model, labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost) 




In [76]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
# 원핫 인코딩, 문자를 숫자로 매핑
input_batch , target_batch = make_batch(seq_data)
for epoch in range(total_epoch):
    _, loss = sess.run([optimizer,cost], feed_dict={X: input_batch, Y:target_batch}) # optimizer 는 학습 , cost는 에포크당 비용 출력
    print("Epoch:", '%04d' % (epoch +1),
         'cost = ', '{:.6f}'.format(loss))
    # model 은 분산과 같은 형태로 예측
    
prediction = tf.cast(tf.argmax(model,1) , tf.int32)
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))  #정확도
input_batch , target_batch =make_batch(seq_data)
predict, accuracy_val = sess.run([prediction, accuracy], feed_dict={X: input_batch, Y: target_batch})
# predict : 수치 > 디코딩
# [ 3  3 15  4  3 11  3  4 18  3]

predict_words =[]
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)

print("\n=== 예측결과 ===")
print(" 입력값 :", [w[:3]+' ' for w in seq_data])
print(" 예측값:" , predict_words)
print("정확도", accuracy_val)

Epoch: 0001 cost =  4.115955
Epoch: 0002 cost =  3.165696
Epoch: 0003 cost =  1.804698
Epoch: 0004 cost =  1.544078
Epoch: 0005 cost =  0.752578
Epoch: 0006 cost =  0.982026
Epoch: 0007 cost =  0.857749
Epoch: 0008 cost =  0.360921
Epoch: 0009 cost =  0.477199
Epoch: 0010 cost =  0.613145
Epoch: 0011 cost =  0.202396
Epoch: 0012 cost =  0.227673
Epoch: 0013 cost =  0.250787
Epoch: 0014 cost =  0.151396
Epoch: 0015 cost =  0.150340
Epoch: 0016 cost =  0.193193
Epoch: 0017 cost =  0.096648
Epoch: 0018 cost =  0.149024
Epoch: 0019 cost =  0.292985
Epoch: 0020 cost =  0.235613
Epoch: 0021 cost =  0.067428
Epoch: 0022 cost =  0.171842
Epoch: 0023 cost =  0.165552
Epoch: 0024 cost =  0.304036
Epoch: 0025 cost =  0.140052
Epoch: 0026 cost =  0.027387
Epoch: 0027 cost =  0.070316
Epoch: 0028 cost =  0.015157
Epoch: 0029 cost =  0.180586
Epoch: 0030 cost =  0.127802

=== 예측결과 ===
 입력값 : ['wor ', 'woo ', 'dee ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
 예측값: ['word', 'wood', 'deep

In [77]:
# 번역 (seq 2 seq )

In [123]:
import tensorflow as tf 
import numpy as np 
from tensorflow.python.framework import ops 
ops.reset_default_graph()

char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz단어놀이나무소녀연습사랑']
num_dic = {n: i for i , n in enumerate(char_arr)}
dic_len = len(num_dic)
print(dic_len)
seq_cata = [['word', '단어'],['wood','나무'],
            ['game', '놀이'],['girl', '소녀'],
            ['test', '연습'],['love', '사랑']]

# input(영어) -> smoking gun,
# output(한글) , target(한글)
# 두 개의 망은 연결


def make_batch(seq_data):
    input_batch = [] 
    output_batch = []
    target_batch =[]
    for seq in seq_data:
        input = [num_dic[n] for n in seq[0]] # 25,17,20,6 > w o r d 
        output = [num_dic[n] for n in ('S' + seq[1])] # start : 비어있음 0 , 29 ,30
        target = [num_dic[n] for n in (seq[1] + 'E')] # 번역의 종료시점[29,30,1]
        input_batch.append(np.eye(dic_len)[input]) # 0000000000010000 > 원핫인코딩
        output_batch.append(np.eye(dic_len)[output])
        target_batch.append(target) # [0,29,30]
    return input_batch, output_batch , target_batch


        
    

41


In [124]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 100 
c_class = n_input = dic_len # 41
enc_input = tf.placeholder(tf.float32, [None,None,n_input]) #  6 (batch size) x 4 (cell) x 41 (input_data)
dec_input = tf.placeholder(tf.float32, [None,None,n_input]) # 6 x 3 x 41
targets = tf.placeholder(tf.int64 , [None,None]) #6 x 3 
# 연결지점을 확인하시오
with tf.variable_scope("encoder"):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden) # 출력차수 128
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell,output_keep_prob=0.5, seed = 100)
    # 셀이 4개로 결정
    outputs,enc_states = tf.nn.dynamic_rnn(enc_cell,enc_input,dtype=tf.float32)

    # output 6 x 4 x 128
with tf.variable_scope("decode"):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden) # 128
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob = 0.5)
    # 셀이 3개로 결정
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input, initial_state = enc_states , dtype=tf.float32) # initial state > 망연결

    # output : 6 x 3 x 128
    
model = tf.layers.dense(outputs,n_class,activation=None) # 6 x 3 x 41
cost = tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels= targets))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

    

In [126]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data)
for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={enc_input: input_batch,
                                  dec_input: output_batch,
                                  targets: target_batch})
    print('Epoch:', '%04d' % (epoch + 1),
          'cost =', '{:.6f}'.format(loss))
                       

Epoch: 0001 cost = 3.825981
Epoch: 0002 cost = 2.708013
Epoch: 0003 cost = 1.502448
Epoch: 0004 cost = 1.036828
Epoch: 0005 cost = 0.498542
Epoch: 0006 cost = 0.450478
Epoch: 0007 cost = 0.375874
Epoch: 0008 cost = 0.228618
Epoch: 0009 cost = 0.222773
Epoch: 0010 cost = 0.192974
Epoch: 0011 cost = 0.181268
Epoch: 0012 cost = 0.076539
Epoch: 0013 cost = 0.128576
Epoch: 0014 cost = 0.081179
Epoch: 0015 cost = 0.052016
Epoch: 0016 cost = 0.055920
Epoch: 0017 cost = 0.039262
Epoch: 0018 cost = 0.013592
Epoch: 0019 cost = 0.051955
Epoch: 0020 cost = 0.030595
Epoch: 0021 cost = 0.008282
Epoch: 0022 cost = 0.007826
Epoch: 0023 cost = 0.009400
Epoch: 0024 cost = 0.004106
Epoch: 0025 cost = 0.021159
Epoch: 0026 cost = 0.024164
Epoch: 0027 cost = 0.029723
Epoch: 0028 cost = 0.012619
Epoch: 0029 cost = 0.004249
Epoch: 0030 cost = 0.005995
Epoch: 0031 cost = 0.007525
Epoch: 0032 cost = 0.002384
Epoch: 0033 cost = 0.014018
Epoch: 0034 cost = 0.001963
Epoch: 0035 cost = 0.003439
Epoch: 0036 cost = 0

In [127]:
def translate(word):
    seq_data = [word, 'P' * len(word)] # 번역된 데이터  (임의의 글자 : 'P')
    input_batch , output_batch, target_batch = make_batch([seq_data])
    prediction = tf.argmax(model,2 ) # 예측값 6x3x41 (41 개의 단어중 하나로 예측)
    result = sess.run(prediction, 
                      feed_dict={enc_input: input_batch ,
                                dec_input: output_batch,
                                targets: target_batch})
    # 번역할 단어에 대한 41개중 하나로 예측
    decoded = [char_arr[i] for i in result[0]] # 숫자를 문자로 매핑
    end = decoded.index('E') # E를 예측
    translated = ''.join(decoded[:end]) # E앞단까지 단어 조합
    return translated

print("word ->", translate("word"))
print("love ->", translate("love"))
print("abcd ->", translate("abcd"))


word -> 단어
love -> 사랑
abcd -> 


In [ ]:
# 문제 : 다음 인코딩 된 값을 2개의 특성을 가진 데이터로 변환
h= [1,1,0,0]
e= [0,1,0,0]
l= [0,0,1,0]
o= [0,0,0,1]
# hellpo 데이터에 댛아ㅕ 개의 특성 추출
# 5x2 
- 배치사이즈 , 셀수 , 셀당아웃풋

In [139]:
ops.reset_default_graph()
from tensorflow.contrib import rnn
import pprint as pp
sess = tf.InteractiveSession()
h= [1,1,0,0]
e= [0,1,0,0]
l= [0,0,1,0]
o= [0,0,0,1]
with tf.variable_scope('hello') as scope:
    hidden_size = 2 
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    x_data = np.array([[h,e,l,l,o]], dtype = np.float32) # 5 x 4 > hello 가 4개로 표현
    print(x_data.shape)
    outputs, states = tf.nn.dynamic_rnn(cell, x_data, dtype= tf.float32)
    sess.run(tf.global_variables_initializer())
    print(outputs.shape)
    print(states.shape)
    pp.pprint(states.eval())



(1, 5, 4)
(1, 5, 2)
(1, 2)
array([[ 0.2724337 , -0.04968274]], dtype=float32)


In [140]:
# 3 batches ' hello', ' eolll' , 'lleel' 2 개의 특성으로 embedding

In [144]:
ops.reset_default_graph()
from tensorflow.contrib import rnn
import pprint as pp
sess = tf.InteractiveSession()
h= [1,1,0,0]
e= [0,1,0,0]
l= [0,0,1,0]
o= [0,0,0,1]
with tf.variable_scope('hello') as scope:
    hidden_size = 2 
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    x_data = np.array([[h,e,l,l,o],[e,o,l,l,l],[l,l,e,e,e]], dtype = np.float32) # 5 x 4 > hello 가 4개로 표현
    print(x_data.shape)
    outputs, states = tf.nn.dynamic_rnn(cell, x_data, dtype= tf.float32)
    sess.run(tf.global_variables_initializer())
    print(outputs.shape)
    print(states.shape)
    pp.pprint(states.eval())


(3, 5, 4)
(3, 5, 2)
(3, 2)
array([[ 0.18328455, -0.54818386],
       [-0.17290401, -0.2799415 ],
       [ 0.17010176,  0.10872702]], dtype=float32)


In [145]:
from konlpy.tag import Okt
okt = Okt()
token = okt.morphs("대한민국의 무궁한 발전과 세계를 이끄는 지도 국가가 되길 희망합니다")
print(token)

['대한민국', '의', '무궁', '한', '발전', '과', '세계', '를', '이끄는', '지도', '국가', '가', '되길', '희망', '합니다']


In [147]:
word2index = {}
for voca in token:
    if voca not in word2index.keys():
        word2index[voca] = len(word2index) # 키 데이터
print(word2index)

def one_hot_encoding(word, word2index):
    one_hot_vector = [0] * (len(word2index))
    index = word2index[word]
    one_hot_vector[index] = 1 
    return one_hot_vector 

one_hot_encoding("대한민국", word2index)

{'대한민국': 0, '의': 1, '무궁': 2, '한': 3, '발전': 4, '과': 5, '세계': 6, '를': 7, '이끄는': 8, '지도': 9, '국가': 10, '가': 11, '되길': 12, '희망': 13, '합니다': 14}


[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]